## Browser Side

In [ ]:
cat hexo/themes/freemind/_config.yml

In [ ]:
cat hexo/themes/freemind/layout/_widget/links.ejs

In [ ]:
cat hexo/themes/freemind/source/js/twitter.js

var twitter = (function() {
  function escapeHtml(str) {
    return $('<div/>').text(str).html();
  }
  function render(target, tweets) {
    var i = 0, fragment = '', t = $(target)[0];

    for(i = 0; i < tweets.length; ++i) {
      fragment += '<li><i class="fa fa-star"></i><a href="https://twitter.com/seii_saintway/status/' + tweets[i].id + '">'
          + tweets[i].created_at.replace('T', '　').replace('.000Z', '') + '</a><p>' + escapeHtml(tweets[i].text||'') + '</p></li>';
    }
    t.innerHTML = fragment;
  }
  return {
    showTweets: function(options) {
      options.blacklist = options.blacklist.split(',');
      $.ajax({
          url: 'https://jhub.name/tweets/?tweet.fields=created_at&max_results=' + options.count
        , dataType: 'json'
        , error: function (err) { $(options.target + ' li.loading').addClass('error').text('Error loading feed'); }
        , success: function (data) {
          if (!data ) { return; }
          var tweets = [];
          for (var i = 0

* [Here](https://github.com/seii-saintway/hexo-theme-freemind/commit/420f1d9c1eda38b000f0b3d3f93b4b143f39ed93) is all the changes at the browser side.

## Server Side

* [Here](https://github.com/zldobbs/alert-mizzou) is a solution using Flask to proxy Tweets API.

In [ ]:
BEARER_TOKEN=''

* https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

In [ ]:
curl -H "Authorization: Bearer ${BEARER_TOKEN}" 'https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=seii_saintway&count=20'

* https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets

In [ ]:
curl -H "Authorization: Bearer ${BEARER_TOKEN}" 'https://api.twitter.com/2/users/807092313298112512/tweets?tweet.fields=created_at&max_results=10'

* My solution using [Traefik](https://doc.traefik.io/traefik/v1.7/basics/#custom-headers) to proxy Tweets API

In [ ]:
cat << EOF | sudo tee /opt/tljh/state/dynamic/tweets.toml > /dev/null

[frontends.tweets]
backend = "tweets"

[frontends.tweets.routes.r1]
rule = "PathPrefix:/tweets/"

[frontends.tweets.routes.r2]
rule = "ReplacePath:/2/users/807092313298112512/tweets"

[frontends.tweets.headers.customrequestheaders]
Authorization = "Bearer ${BEARER_TOKEN}"

[frontends.tweets.headers.customresponseheaders]
Access-Control-Allow-Origin = "*"

[backends.tweets.servers.s1]
url = "https://api.twitter.com"
weight = 1

EOF

In [ ]:
sudo systemctl restart traefik

In [ ]:
curl 'https://jhub.name/tweets/?tweet.fields=created_at&max_results=10'